## Olympics environnemental impact analysis

In [26]:
# let's scrape some data from the web
# https://stats.oecd.org/Index.aspx?DataSetCode=air_ghg

Scraping data from : https://stats.oecd.org/Index.aspx?DataSetCode=air_ghg

In [27]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm as tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [28]:
driver = webdriver.Firefox()
driver.get("https://stats.oecd.org/Index.aspx?DataSetCode=air_ghg")

Drop-down menu :

In [29]:
menu = '//*[@id="PDim_VAR"]' 
element = driver.find_element(By.XPATH,menu).click()

Choosing the emmission sector :

In [30]:
sleep(2)
npage = 2
xpath_expression = "//select/option[@value ='2~ENER_IND']" # 0~GHG
sector = driver.find_element(By.XPATH, xpath_expression).click()   

BeautifulSoup to parse the html : 

In [31]:
soup = bs(driver.page_source, 'html.parser')
table = soup.find('table', {'class': 'DataTable'})
# we want to find all tr in the table with an id starting with row
rows = table.findAll('tr', {'id': lambda L: L and L.startswith('row')})
# we display the first rows
rows[:5]

[<tr class="row1" id="row1"><td class="RowDimLabel" colspan="2">Australia</td><td class="RowDimLabel"></td><td class="Data">143 172.76</td><td class="Data">146 396.55</td><td class="Data">149 719.80</td><td class="Data">151 492.72</td><td class="Data">152 307.64</td><td class="Data">158 140.20</td><td class="Data">162 742.64</td><td class="Data">169 413.37</td><td class="Data">182 065.82</td><td class="Data">189 654.80</td><td class="Data">192 519.66</td><td class="Data">200 056.94</td><td class="Data">202 550.84</td><td class="Data">205 188.59</td><td class="Data">214 175.42</td><td class="Data">216 528.07</td><td class="Data">221 026.74</td><td class="Data">224 087.96</td><td class="Data">225 886.85</td><td class="Data">232 820.84</td><td class="Data">226 959.03</td><td class="Data">221 003.02</td><td class="Data">222 772.02</td><td class="Data">211 608.43</td><td class="Data">205 575.02</td><td class="Data">212 312.51</td><td class="Data">219 696.63</td><td class="Data">218 631.00</

Creation of the dataframe : 

In [32]:
df = pd.DataFrame(columns=['sector', 'country', 'year', 'GH emissions'])

Test of data extraction for the first sector : Energy Industries

In [33]:
# we extract the data from rows and add them to the dataframe df
for row in tqdm.tqdm(rows):
    cells = row.findAll('td')
    sector = '1A1 - Energy Industries'
    country = cells[0].text
    for i in range(0, 32):
        year = 1990 + i
        GH = cells[i+2].text
        df = pd.concat([df, pd.DataFrame([[sector, country, year, GH]], columns=['sector', 'country', 'year', 'GH emissions'])])

100%|██████████| 64/64 [00:00<00:00, 84.46it/s]


In [34]:
# we print the head of the dataframe
df.head(10)

,sector,country,year,GH emissions
0,1A1 - Energy Industries,Australia,1990,143 172.76
0,1A1 - Energy Industries,Australia,1991,146 396.55
0,1A1 - Energy Industries,Australia,1992,149 719.80
0,1A1 - Energy Industries,Australia,1993,151 492.72
0,1A1 - Energy Industries,Australia,1994,152 307.64
0,1A1 - Energy Industries,Australia,1995,158 140.20
0,1A1 - Energy Industries,Australia,1996,162 742.64
0,1A1 - Energy Industries,Australia,1997,169 413.37
0,1A1 - Energy Industries,Australia,1998,182 065.82
0,1A1 - Energy Industries,Australia,1999,189 654.80


In [35]:
# npage += 1
# xpath_expression = f"//select/option[contains(@value, '{npage}~ENER_')]"
# sector = driver.find_element(By.XPATH, xpath_expression).click() 

### Automatisation of the scraping process for the OECD website

Function to scrape the data from the website according to the sector chosen : 

In [36]:
# now let's do a function to extract the data from the website starting from npage 2 to npage 8
def extract_data(npage, sector_name, df):
    menu = '//*[@id="PDim_VAR"]' 
    driver.find_element(By.XPATH,menu).click() # menu

    xpath_expression = f"//select/option[contains(@value, '{npage}~ENER_')]"
    driver.find_element(By.XPATH, xpath_expression).click() # page of the secteur
    sleep(7)

    soup = bs(driver.page_source, 'html.parser')
    table = soup.find('table', {'class': 'DataTable'})
    rows = table.findAll('tr', {'id': lambda L: L and L.startswith('row')})

    for row in tqdm.tqdm(rows):
        cells = row.findAll('td')
        country = cells[0].text
        if country == 'Non-OECD Economies':
            country = cells[1].text
            for i in range(0, 32):
                year = 1990 + i
                GH = cells[i+3].text
                df = pd.concat([df, pd.DataFrame([[sector_name, country, year, GH]], columns=['sector', 'country', 'year', 'GH emissions'])])
        else :
            for i in range(0, 32):
                year = 1990 + i
                GH = cells[i+2].text
                df = pd.concat([df, pd.DataFrame([[sector_name, country, year, GH]], columns=['sector', 'country', 'year', 'GH emissions'])])
            
    return df

Extracting the name of the sectors :

In [37]:
# we extract the names of the sectors :
sleep(7)
sectors = []
for i in range(2, 9):
    xpath_expression = f"//select/option[contains(@value, '{i}~ENER_')]"
    sector = driver.find_element(By.XPATH, xpath_expression)
    sectors.append(sector.text)
sectors

['    1A1 - Energy Industries',
 '    1A2 - Manufacturing industries and construction',
 '    1A3 - Transport',
 '    1A4 - Residential and other sectors',
 '    1A5 - Energy - Other',
 '    1B - Fugitive Emissions from Fuels',
 '    1C - CO2 from Transport and Storage']

In [38]:
# in '    1A1 - Energy Industries', we want to keep only 'Energy Industries'. We do that for all the sectors
sectors = [sector.split(' - ')[1] for sector in sectors]
# the sector 'Energy' is re-changed to 'Energy - Other'
sectors[4] = 'Energy - Other'
sectors

['Energy Industries',
 'Manufacturing industries and construction',
 'Transport',
 'Residential and other sectors',
 'Energy - Other',
 'Fugitive Emissions from Fuels',
 'CO2 from Transport and Storage']

In [39]:
# Cleaning the dataframe ...
df = pd.DataFrame(columns=['sector', 'country', 'year', 'GH emissions'])
df

,sector,country,year,GH emissions


In [40]:
# we extract the data from npage 2 to npage 8
for i in tqdm.tqdm(range(0, 7)):
    df = extract_data(i+2, sectors[i], df)

100%|██████████| 7/7 [00:58<00:00,  8.31s/it]


In [41]:
df.head()

,sector,country,year,GH emissions
0,Energy Industries,Australia,1990,143 172.76
0,Energy Industries,Australia,1991,146 396.55
0,Energy Industries,Australia,1992,149 719.80
0,Energy Industries,Australia,1993,151 492.72
0,Energy Industries,Australia,1994,152 307.64


In [48]:
# we search for the lines that have both country = 'OECD - Europe' and sector = 'Fugitive Emissions from Fuels'
df[(df['country'] == 'OECD - Europe') & (df['sector'] == 'Fugitive Emissions from Fuels')].head(10)

,sector,country,year,GH emissions
0,Fugitive Emissions from Fuels,OECD - Europe,1990,184 067.20
0,Fugitive Emissions from Fuels,OECD - Europe,1991,175 467.25
0,Fugitive Emissions from Fuels,OECD - Europe,1992,172 353.00
0,Fugitive Emissions from Fuels,OECD - Europe,1993,170 686.44
0,Fugitive Emissions from Fuels,OECD - Europe,1994,158 913.91
0,Fugitive Emissions from Fuels,OECD - Europe,1995,159 189.67
0,Fugitive Emissions from Fuels,OECD - Europe,1996,154 823.81
0,Fugitive Emissions from Fuels,OECD - Europe,1997,150 382.61
0,Fugitive Emissions from Fuels,OECD - Europe,1998,143 130.36
0,Fugitive Emissions from Fuels,OECD - Europe,1999,139 553.66


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11680 entries, 0 to 0
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sector        11680 non-null  object
 1   country       11680 non-null  object
 2   year          11680 non-null  object
 3   GH emissions  11680 non-null  object
dtypes: object(4)
memory usage: 456.2+ KB


In [50]:
# let's crate a checkpoint for the dataframe df
df.to_csv('data_csv/cp_GH_emissions.csv', index=False)

In [95]:
# we modify a copy of the dataframe imported from the checkpoint
# we import data from the checkpoint 'data_csv/cp_GH_emissions.csv'
df2 = pd.read_csv('data_csv/cp_GH_emissions.csv')
df2.head()

,sector,country,year,GH emissions
0,Energy Industries,Australia,1990,143 172.76
1,Energy Industries,Australia,1991,146 396.55
2,Energy Industries,Australia,1992,149 719.80
3,Energy Industries,Australia,1993,151 492.72
4,Energy Industries,Australia,1994,152 307.64


In [96]:
# we convert sector and country to string, year to int and GH emissions to float GH emmissions are currently of the form '123 313.04'"Web Scrapping CM1.docx"
df2['sector'] = df2['sector'].astype(str)
df2['country'] = df2['country'].astype(str)
df2['year'] = df2['year'].astype(int)
df2['GH emissions'] = df2['GH emissions'].astype(str) 

In [97]:
df2['GH emissions'] = df2['GH emissions'].replace("..", 'NaN')
df2['GH emissions'] = df2['GH emissions'].str.replace('(B)', '')
df2['GH emissions'] = df2['GH emissions'].str.replace('\xa0', '')

In [98]:
df2['GH emissions'] = df2['GH emissions'].apply(lambda x: float(x) if x != '' else np.nan)

In [99]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11680 entries, 0 to 11679
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sector        11680 non-null  object 
 1   country       11680 non-null  object 
 2   year          11680 non-null  int32  
 3   GH emissions  10268 non-null  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 319.5+ KB
